In [119]:
X_train = open('X_train.txt')
line = X_train.readline()
train = list()
while line:
    line = line.strip('\n')
    line = line.split()
    train.append(line)
    line = X_train.readline()
print(type(train))
print(len(train))
print(type(train[0]))

<class 'list'>
341072
<class 'list'>


In [9]:
from keras.layers.core import Activation, Dense
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
import nltk 
import collections
import numpy as np

Using TensorFlow backend.
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6

In [18]:
y_train = open('y_train.txt')
label = list()
line = y_train.readline()
while line:
    line = line.strip('\n')
    line = line.split()
    label.append(int(line[0]))
    line = y_train.readline()

In [21]:
import numpy as np
label = np.array(label)
train = np.array(train)

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train)

In [23]:
vocab=tokenizer.word_index

In [24]:
len(vocab)

383395

In [26]:
encoded_train = tokenizer.texts_to_sequences(train)

In [27]:
type(encoded_train)

list

In [28]:
encoded_train = np.array(encoded_train)

In [30]:
temp_len = 0
for i in range(len(encoded_train)):
    temp_len += len(encoded_train[i])

mean_len = temp_len / len(encoded_train)
mean_len = int(mean_len)

encoded_train = sequence.pad_sequences(encoded_train, maxlen=mean_len)

In [99]:
label = label.reshape(-1,1)

In [100]:
label.shape

(341072, 1)

In [101]:
import keras
label_train = keras.utils.to_categorical(label)

In [111]:
len(vocab)

383395

In [116]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras import layers
from keras.optimizers import Adam
from keras.models import Model,Sequential
import keras
embedding_dim = 128
model = Sequential()
model.add(layers.Embedding(len(vocab)+1, embedding_dim,input_length=mean_len))
model.add(LSTM(50))
model.add(Dense(6,activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 80, 128)           49074688  
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 76, 128)           82048     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 774       
Total params: 49,157,510
Trainable params: 49,157,510
Non-trainable params: 0
_________________________________________________________________
None


In [117]:
adam = keras.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(encoded_train, label_train,epochs=1,verbose=1,batch_size=500)

Epoch 1/1
341072/341072 [==============================] - 494s 1ms/step - loss: 0.9532 - acc: 0.6643


In [120]:
X_test = open('X_test.txt')
line = X_test.readline()
test = list()
while line:
    line = line.strip('\n')
    line = line.split()
    test.append(line)
    line = X_test.readline()
print(type(test))
print(len(test))
print(type(test[0]))

<class 'list'>
113691
<class 'list'>


In [138]:
y_test = open('y_test.txt')
label_test = list()
line = y_test.readline()
while line:
    line = line.strip('\n')
    line = line.split()
    label_test.append(int(line[0]))
    line = y_test.readline()

In [122]:
encoded_test = tokenizer.texts_to_sequences(test)
encoded_test = np.array(encoded_test)
encoded_test = sequence.pad_sequences(encoded_test, maxlen=mean_len)

In [141]:
predict_test = model.predict(encoded_test)

In [142]:
acc_num = 0
for i in range(len(encoded_test)):
    cla = np.argmax(predict_test[i])
    if cla == label_test[i]:
        acc_num = acc_num + 1

print(acc_num/len(encoded_test))

0.7049810451135095
